In [3]:
import numpy as np
from scipy import signal
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.preprocessing import LabelBinarizer
import csv
import pickle
import datetime
from sklearn.preprocessing import MinMaxScaler

In [168]:
# set path to datasets
datapath = '/home/jingyan/Documents/ME499-WinterProject/mindwave/src/motor_cortex_ml/data/'
def load_record(sec,runnum=7):
    for secnum in sec:
        for run in range(1,runnum): #6runs for each session
            savetag='_exp15_sec'+str(secnum)+'_run'+str(run)
            csvname =datapath + 'record'+savetag+'.csv'
            data = np.loadtxt(csvname,delimiter = ',')
            if run==1 and secnum==sec[0]:
                x = data[:,:-1]
                y = data[:,-1]
            else:
                x=np.append(x,data[:,:-1],axis=0)
                y=np.append(y,data[:,-1],axis=0)
    return x, y

def fft(data, fs):
    L = len(data)
    freq = np.linspace(0.0, 1.0 / (2.0 * fs **-1), int(L / 2))
    yi = np.fft.fft(data)#[1:]
    y = yi[range(int(L / 2))]
    # ysample= (2.0*abs(yi/L))[range(int(L/2))]
    return freq, abs(y)
def drop_class(data,labels,remove_list=[0]):
    keep_list=np.isin(labels,remove_list,invert=True)
    label_keep=labels[keep_list]
    data_keep=data[keep_list]
    return data_keep,label_keep
def set_baseline_ref(data,labels,base_class=[0]):
    base_list=np.isin(labels,base_class,invert=False)
    baseline_ffts=data[base_list]
    baseline_fft_ref=np.mean(baseline_ffts,axis=0)
    data_keep,label_keep=drop_class(data,labels)
    if data_keep.shape[1]!=baseline_fft_ref.shape[0]: ValueError('size mismatch')
    data_ref=(data_keep-baseline_fft_ref)/baseline_fft_ref
    return data_ref,label_keep,baseline_fft_ref    

channel_num=8
chunk=60 #number of trails for each task in one session
fs=250
band=(7,13)
band2=(18,30)
notchval=60
samplesize=7.5*250
train_data,train_label=load_record([1],3)
# test_data,test_label=load_record([4],3)

In [49]:
train_data.shape

(9000, 24)

In [169]:
train_keep,label_keep,baseline_fft_ref=set_baseline_ref(train_data,train_label)
scaler=MinMaxScaler()
scaled_train_data=scaler.fit_transform(train_keep)
from sklearn.model_selection import train_test_split

np.savetxt("baseline_ref.csv", baseline_fft_ref, delimiter=",")

x_train,x_test,y_train,y_test=train_test_split(train_keep,label_keep,test_size=0.25)
print(x_train.shape,y_train.shape)

((4500, 24), (4500,))


In [170]:
#Linear SVM classifier
clf1 = LinearSVC(C = 0.1, intercept_scaling=1, loss='hinge', max_iter=1000,multi_class='ovr', penalty='l2', random_state=1, tol=0.00001)
clf1.fit(x_train,y_train)
score_train1=clf1.score(x_train,y_train)
score_test1=clf1.score(x_test,y_test)
print(score_train1,score_test1)

(0.6266666666666667, 0.6226666666666667)


In [180]:
model=KNeighborsClassifier(n_neighbors=5)
model.fit(x_train,y_train)
score_train=model.score(x_train,y_train)
score_test=model.score(x_test,y_test)
print(score_train,score_test)
model_filename='mindwave_model.pkl'
with open(model_filename,'wb') as file:
    pickle.dump(model,file)

(0.8873333333333333, 0.8326666666666667)


In [172]:
model=GradientBoostingClassifier()
model.fit(x_train,y_train)
score_train=model.score(x_train,y_train)
score_test=model.score(x_test,y_test)
print(score_train,score_test)
model_filename='mindwave_model.pkl'
with open(model_filename,'wb') as file:
    pickle.dump(model,file)

(0.8297777777777777, 0.7453333333333333)


In [173]:
model=AdaBoostClassifier(n_estimators=100)#OneVsRestClassifier(SVC(kernel='linear'))
model.fit(x_train,y_train)
score_train=model.score(x_train,y_train)
score_test=model.score(x_test,y_test)
print(score_train,score_test)
# model_filename='mindwave_model.pkl'
# with open(model_filename,'wb') as file:
#     pickle.dump(model,file)

(0.7344444444444445, 0.6626666666666666)


In [174]:
model=DecisionTreeClassifier()
model.fit(x_train,y_train)
score_train=model.score(x_train,y_train)
score_test=model.score(x_test,y_test)
print(score_train,score_test)
model_filename='mindwave_model.pkl'
with open(model_filename,'wb') as file:
    pickle.dump(model,file,protocol=2)

(1.0, 0.736)


In [175]:
model=LinearDiscriminantAnalysis()
model.fit(x_train,y_train)
score_train=model.score(x_train,y_train)
score_test=model.score(x_test,y_test)
print(score_train,score_test)
# model_filename='mindwave_model.pkl'
# with open(model_filename,'wb') as file:
#     pickle.dump(model,file)

(0.6111111111111112, 0.6113333333333333)


In [176]:
model2 = SVC(20,'rbf', degree=10, gamma='auto', coef0=0.0, tol=0.001, cache_size=10000, max_iter=-1, decision_function_shape='ovr')
model2.fit(x_train,y_train)
score_train2=model2.score(x_train,y_train)
score_test2=model2.score(x_test,y_test)
print(score_train2,score_test2)

(0.8504444444444444, 0.8193333333333334)
